In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = ''
resume = './log/star/128/b1/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 280
test_batch = 200
lr = 0.04
schedule = [75, 150, 225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b1' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [16 | 300] LR: 0.292000
1/981 Data:1.467 | Batch:5.311 | Total:0:00:05 | ETA:1:26:45 | Loss:0.4004509747028351 | top1:83.21428680419922
11/981 Data:0.002 | Batch:0.609 | Total:0:00:10 | ETA:0:16:20 | Loss:0.3035276532173157 | top1:86.85064697265625
21/981 Data:0.001 | Batch:0.500 | Total:0:00:16 | ETA:0:08:52 | Loss:0.301321226926077 | top1:87.09183502197266
31/981 Data:0.001 | Batch:0.490 | Total:0:00:21 | ETA:0:08:23 | Loss:0.29525673197161767 | top1:87.35023498535156
41/981 Data:0.001 | Batch:0.502 | Total:0:00:26 | ETA:0:07:56 | Loss:0.2984246502562267 | top1:87.14285278320312
51/981 Data:0.001 | Batch:0.492 | Total:0:00:31 | ETA:0:07:56 | Loss:0.30069950515148686 | top1:87.05883026123047
61/981 Data:0.001 | Batch:0.588 | Total:0:00:36 | ETA:0:07:57 | Loss:0.30157409193085843 | top1:87.16627502441406
71/981 Data:0.001 | Batch:0.502 | Total:0:00:42 | ETA:0:08:08 | Loss:0.29996171383790565 | top1:87.26860809326172
81/981 Data:0.000 | Batch:0.494 | Total:0:00:47 | ETA:0:07:26 

721/981 Data:0.001 | Batch:0.679 | Total:0:06:53 | ETA:0:02:48 | Loss:0.3006895881833382 | top1:87.14682006835938
731/981 Data:0.000 | Batch:0.670 | Total:0:06:59 | ETA:0:02:42 | Loss:0.30032277037931043 | top1:87.16093444824219
741/981 Data:0.003 | Batch:0.980 | Total:0:07:08 | ETA:0:03:20 | Loss:0.30020689801407247 | top1:87.16213989257812
751/981 Data:0.001 | Batch:0.964 | Total:0:07:17 | ETA:0:03:31 | Loss:0.3001006565462257 | top1:87.17139434814453
761/981 Data:0.001 | Batch:0.952 | Total:0:07:26 | ETA:0:03:33 | Loss:0.30002121868177406 | top1:87.16539001464844
771/981 Data:0.001 | Batch:0.978 | Total:0:07:36 | ETA:0:03:24 | Loss:0.29981875744311265 | top1:87.16740417480469
781/981 Data:0.001 | Batch:0.952 | Total:0:07:46 | ETA:0:03:15 | Loss:0.30032153112787596 | top1:87.13965606689453
791/981 Data:0.001 | Batch:0.609 | Total:0:07:55 | ETA:0:02:57 | Loss:0.3002966914782783 | top1:87.13924407958984
801/981 Data:0.000 | Batch:0.496 | Total:0:08:01 | ETA:0:02:03 | Loss:0.29997249120

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


153/153 Data:0.001 | Batch:0.498 | Total:0:00:33 | ETA:0:00:00 | Loss:0.2295368222195074 | top1:90.6454849243164

Epoch: [17 | 300] LR: 0.068000
1/981 Data:0.852 | Batch:1.406 | Total:0:00:01 | ETA:0:22:58 | Loss:0.28661108016967773 | top1:87.5
11/981 Data:0.000 | Batch:0.698 | Total:0:00:07 | ETA:0:11:32 | Loss:0.25949000770395453 | top1:89.44805145263672
21/981 Data:0.000 | Batch:0.692 | Total:0:00:14 | ETA:0:10:21 | Loss:0.2504281472592127 | top1:89.76190948486328
31/981 Data:0.001 | Batch:0.697 | Total:0:00:20 | ETA:0:10:20 | Loss:0.24611717799017507 | top1:89.74654388427734
41/981 Data:0.000 | Batch:0.491 | Total:0:00:27 | ETA:0:10:11 | Loss:0.23760288663026763 | top1:90.25260925292969
51/981 Data:0.000 | Batch:0.661 | Total:0:00:33 | ETA:0:09:33 | Loss:0.23877287845985562 | top1:90.15406799316406
61/981 Data:0.000 | Batch:0.693 | Total:0:00:39 | ETA:0:09:56 | Loss:0.23654169186216886 | top1:90.24590301513672
71/981 Data:0.000 | Batch:0.682 | Total:0:00:46 | ETA:0:09:47 | Loss:0.2

711/981 Data:0.000 | Batch:0.687 | Total:0:08:27 | ETA:0:02:51 | Loss:0.2087511323947779 | top1:91.36528778076172
721/981 Data:0.000 | Batch:0.717 | Total:0:08:34 | ETA:0:02:48 | Loss:0.20857773854522532 | top1:91.37012481689453
731/981 Data:0.000 | Batch:0.686 | Total:0:08:41 | ETA:0:02:44 | Loss:0.2081263388536371 | top1:91.38948059082031
741/981 Data:0.001 | Batch:0.499 | Total:0:08:47 | ETA:0:02:36 | Loss:0.2082061037903855 | top1:91.38423156738281
751/981 Data:0.000 | Batch:0.698 | Total:0:08:53 | ETA:0:02:24 | Loss:0.20803459385501719 | top1:91.3876724243164
761/981 Data:0.000 | Batch:0.695 | Total:0:09:00 | ETA:0:02:23 | Loss:0.20780828571664206 | top1:91.39478302001953
771/981 Data:0.000 | Batch:0.672 | Total:0:09:06 | ETA:0:02:16 | Loss:0.20787495138889467 | top1:91.38734436035156
781/981 Data:0.000 | Batch:0.694 | Total:0:09:13 | ETA:0:02:09 | Loss:0.2077837116281751 | top1:91.39244842529297
791/981 Data:0.000 | Batch:0.532 | Total:0:09:19 | ETA:0:02:02 | Loss:0.2075612587588

...

531/981 Data:0.002 | Batch:0.978 | Total:0:05:58 | ETA:0:05:49 | Loss:0.08709580737487072 | top1:96.66195678710938
541/981 Data:0.001 | Batch:0.543 | Total:0:06:05 | ETA:0:05:33 | Loss:0.08765634132098138 | top1:96.64311981201172
551/981 Data:0.000 | Batch:0.670 | Total:0:06:11 | ETA:0:04:24 | Loss:0.08752448912946585 | top1:96.6502456665039
561/981 Data:0.007 | Batch:0.661 | Total:0:06:17 | ETA:0:04:08 | Loss:0.08727683621639662 | top1:96.6603012084961
571/981 Data:0.001 | Batch:0.525 | Total:0:06:23 | ETA:0:04:21 | Loss:0.08742811401650283 | top1:96.66124725341797
581/981 Data:0.001 | Batch:0.652 | Total:0:06:29 | ETA:0:04:00 | Loss:0.08751627024449088 | top1:96.65970611572266
591/981 Data:0.001 | Batch:0.643 | Total:0:06:35 | ETA:0:03:51 | Loss:0.08745978783355954 | top1:96.65880584716797
601/981 Data:0.001 | Batch:0.533 | Total:0:06:41 | ETA:0:03:54 | Loss:0.08742112212019434 | top1:96.65498352050781
611/981 Data:0.001 | Batch:0.627 | Total:0:06:47 | ETA:0:03:40 | Loss:0.0873736102

261/981 Data:0.017 | Batch:0.986 | Total:0:02:50 | ETA:0:10:59 | Loss:0.0927323695143749 | top1:96.36015319824219
271/981 Data:0.002 | Batch:0.924 | Total:0:02:59 | ETA:0:10:58 | Loss:0.09313457661284291 | top1:96.34159088134766
281/981 Data:0.001 | Batch:0.906 | Total:0:03:06 | ETA:0:08:06 | Loss:0.09284710670652338 | top1:96.35104370117188
291/981 Data:0.001 | Batch:0.883 | Total:0:03:14 | ETA:0:09:56 | Loss:0.09252769740688842 | top1:96.3672103881836
301/981 Data:0.003 | Batch:0.641 | Total:0:03:21 | ETA:0:07:13 | Loss:0.09284877175607555 | top1:96.3609390258789
311/981 Data:0.009 | Batch:0.555 | Total:0:03:26 | ETA:0:06:29 | Loss:0.09294534875888533 | top1:96.35277557373047
321/981 Data:0.001 | Batch:0.629 | Total:0:03:33 | ETA:0:07:00 | Loss:0.09280454262951825 | top1:96.37071990966797
331/981 Data:0.001 | Batch:0.634 | Total:0:03:39 | ETA:0:06:18 | Loss:0.09275790757508076 | top1:96.36814880371094
341/981 Data:0.001 | Batch:0.646 | Total:0:03:44 | ETA:0:06:10 | Loss:0.09282622865

153/153 Data:0.002 | Batch:0.124 | Total:0:00:29 | ETA:0:00:00 | Loss:0.0568115933142788 | top1:97.84732055664062

Epoch: [101 | 300] LR: 0.027432
1/981 Data:1.133 | Batch:2.106 | Total:0:00:02 | ETA:0:34:25 | Loss:0.11595024168491364 | top1:95.71428680419922
11/981 Data:0.006 | Batch:0.919 | Total:0:00:10 | ETA:0:16:08 | Loss:0.09245778484777971 | top1:96.13636016845703
21/981 Data:0.001 | Batch:0.805 | Total:0:00:19 | ETA:0:13:43 | Loss:0.08868720293754623 | top1:96.37755584716797
31/981 Data:0.001 | Batch:0.853 | Total:0:00:28 | ETA:0:13:53 | Loss:0.08845837822844906 | top1:96.48617553710938
41/981 Data:0.003 | Batch:0.739 | Total:0:00:34 | ETA:0:10:42 | Loss:0.08600586016730564 | top1:96.59407806396484
51/981 Data:0.017 | Batch:0.575 | Total:0:00:42 | ETA:0:11:52 | Loss:0.08824427016809874 | top1:96.56163024902344
61/981 Data:0.001 | Batch:0.639 | Total:0:00:50 | ETA:0:11:51 | Loss:0.088281514458969 | top1:96.59835815429688
71/981 Data:0.001 | Batch:0.639 | Total:0:00:56 | ETA:0:09

711/981 Data:0.001 | Batch:0.614 | Total:0:07:40 | ETA:0:02:36 | Loss:0.08850170112622355 | top1:96.58077239990234
721/981 Data:0.001 | Batch:0.506 | Total:0:07:46 | ETA:0:02:41 | Loss:0.08857462279519245 | top1:96.57618713378906
731/981 Data:0.001 | Batch:0.599 | Total:0:07:52 | ETA:0:02:23 | Loss:0.08867476892016736 | top1:96.57122802734375
741/981 Data:0.001 | Batch:0.504 | Total:0:07:58 | ETA:0:02:22 | Loss:0.08855136584203976 | top1:96.57605743408203
751/981 Data:0.001 | Batch:0.615 | Total:0:08:04 | ETA:0:02:22 | Loss:0.08856239788154867 | top1:96.57646942138672
761/981 Data:0.001 | Batch:0.878 | Total:0:08:12 | ETA:0:02:47 | Loss:0.0884276986361999 | top1:96.58110046386719
771/981 Data:0.001 | Batch:0.827 | Total:0:08:21 | ETA:0:03:08 | Loss:0.08827827665254222 | top1:96.58930969238281
781/981 Data:0.001 | Batch:0.787 | Total:0:08:30 | ETA:0:02:58 | Loss:0.08812204634630039 | top1:96.59593963623047
791/981 Data:0.001 | Batch:0.900 | Total:0:08:39 | ETA:0:02:52 | Loss:0.089253235

441/981 Data:0.001 | Batch:0.660 | Total:0:04:49 | ETA:0:05:37 | Loss:0.08695003906446519 | top1:96.6026840209961
451/981 Data:0.001 | Batch:0.656 | Total:0:04:55 | ETA:0:05:18 | Loss:0.08746768098877962 | top1:96.57427978515625
461/981 Data:0.001 | Batch:0.648 | Total:0:05:01 | ETA:0:05:08 | Loss:0.0877121888392013 | top1:96.5703353881836
471/981 Data:0.001 | Batch:0.590 | Total:0:05:06 | ETA:0:05:08 | Loss:0.08784663320505189 | top1:96.55899810791016
481/981 Data:0.001 | Batch:0.644 | Total:0:05:13 | ETA:0:05:14 | Loss:0.08780484522640582 | top1:96.56222534179688
491/981 Data:0.001 | Batch:0.897 | Total:0:05:21 | ETA:0:06:09 | Loss:0.08782301278682444 | top1:96.56604766845703
501/981 Data:0.001 | Batch:0.968 | Total:0:05:30 | ETA:0:07:12 | Loss:0.08790663402237578 | top1:96.56829071044922
511/981 Data:0.002 | Batch:0.920 | Total:0:05:39 | ETA:0:07:12 | Loss:0.08777894949405628 | top1:96.57814025878906
521/981 Data:0.001 | Batch:0.914 | Total:0:05:48 | ETA:0:06:59 | Loss:0.08774347536